# Import

In [1]:
import re
import sys

sys.path.append("../")
import json
import warnings
from string import Template
from time import sleep

import matplotlib.pyplot as plt
import pandas as pd
import requests
from tqdm import tqdm
from vook_db_lambda.config import MAX_PAGE, REQ_URL, WANT_ITEMS_RAKUTEN, req_params


Bad key ; font.family in file /Users/ozawaatsushi/.matplotlib/matplotlibrc, line 262 ('; font.family:  IPAexGothic')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.9.0/lib/matplotlib/mpl-data/matplotlibrc
or from the matplotlib source distribution


Now: Local env


In [2]:
df = pd.read_excel(
    "../data/input/相場ランキング_animeTシャツ_hypebeast紹介アイテム.xlsx",
    sheet_name="raw",
    header=None,
)

In [3]:
df.columns = ["anime", "output_text"]
df = df[["シュプリーム" not in txt for txt in df["output_text"]]]
df = df[["supreme" not in txt for txt in df["output_text"]]]
df = df[["SUPREME" not in txt for txt in df["output_text"]]]
df = df[["Supreme" not in txt for txt in df["output_text"]]]

df["price"] = [
    int(
        re.findall(
            r"\d+",
            txt.split(" ")[0]
            .replace(",", "")
            .replace("2点おまとめ", "")
            .replace("3日間", "")
            .replace("778", "")
            .replace("3月29日", "")
            .replace("01'y", "")
            .replace("1990s", "")
            .replace("1988年", "")
            .replace("1997年", "")
            .replace("2002年", "")
            .replace("2002年製", "")
            .replace("2002's", "")
            .replace("2000s", "")
            .replace("2008", "")
            .replace("230624③", "")
            .replace("00s", "")
            .replace("00's", "")
            .replace("00S", "")
            .replace("20s", "")
            .replace("90s", "")
            .replace("90's", "")
            .replace("90'S", "")
            .replace("90‘ｓ", "")
            .replace("80s", "")
            .replace("80's", "")
            .replace("70s", "")
            .replace("70's", ""),
        )[0]
    )
    for txt in df["output_text"]
]
# df[df["price"] >= 100000].sort_values("price", ascending=False)
df_true_more_than_10000 = df[df["price"] < 10000].copy()  # 00s
df_others = df[df["price"] >= 10000].copy()
df_true_more_than_10000["price"] = df_true_more_than_10000["price"] * 100
df_mst = pd.concat([df_others, df_true_more_than_10000])

In [4]:
assert not df_mst[df_mst["price"] < 10000].shape[0]

In [5]:
df_anime_price = (
    df_mst[["anime", "price"]]
    .groupby("anime")
    .median()["price"]
    .apply(lambda x: round(x))
    .reset_index()
)
df_anime_nuniq = (
    df_mst[["anime", "output_text"]].groupby("anime").nunique().reset_index()
)
df_agg_tmp = pd.merge(df_anime_nuniq, df_anime_price, how="inner", on="anime")

In [6]:
df_agg_tmp.sort_values(["output_text"], ascending=False)

,anime,output_text,price
0,AKIRA,230,21500
8,ドラゴンボール,120,18300
11,攻殻機動隊,116,19000
9,ナルト,74,18000
10,マッハGOGOGO,22,20400
12,犬夜叉,22,20000
13,鋼の錬金術師,15,20000
7,ジョジョの奇妙な冒険,10,12320
2,SLAM DUNK,8,18500
3,うる星やつら,5,13799
